In [7]:
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
import os

Settings.embed_model =  OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
)

Settings.llm = Gemini(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY"),  
)

load_dotenv()

C:\Users\ijona\AppData\Local\Temp\ipykernel_22332\2237849840.py:15: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  Settings.llm = Gemini(


True

In [9]:
docs = SimpleDirectoryReader(
    input_files=[
        r"C:\Users\ijona\Desktop\DocumentIngestionPipeline-AllCognix\samples\uber_2021.pdf",
        r"C:\Users\ijona\Desktop\DocumentIngestionPipeline-AllCognix\samples\paul_graham_essay.txt",
        r"C:\Users\ijona\Desktop\DocumentIngestionPipeline-AllCognix\samples\RAFT.pdf"
]
).load_data()

In [10]:
print("Total Documents:",len(docs))

Total Documents: 320


In [11]:
print("Document ID:", docs[0].doc_id)

Document ID: 6751cdd3-3a66-43a8-92d8-77c2fe36d7cc


In [ ]:
collection_name = os.getenv("MILVUS_COLLECTION") or "allcognix"

vector_store = MilvusVectorStore(
    uri=os.getenv("MILVUS_ENDPOINT"),
    token=os.getenv("MILVUS_TOKEN"),
    collection_name=collection_name,
    dim=768,
    # overwrite=True, # drop the collection if it already exists
    enable_sparse=True  # enable the default full-text search using BM25
)

2026-01-11 19:19:08,884 - WARNING - Sparse embedding function is not provided, using default.
2026-01-11 19:19:08,889 - WARNING - No built-in function detected, using BGEM3SparseEmbeddingFunction().
Fetching 30 files:  60%|██████    | 18/30 [00:21<00:02,  4.57it/s]

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    docs[:20], storage_context=storage_context, show_progress=True
)

2026-01-11 19:00:51,297 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-11 19:00:51,962 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-11 19:00:52,726 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
2026-01-11 19:00:53,148 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


In [ ]:
query_engine = index.as_query_engine(
    vector_store_query_mode="hybrid", 
    similarity_top_k=5
)
response = query_engine.query("Tell me about Uber")
print(textwrap.fill(str(response), 100))

2026-01-11 19:06:17,696 - INFO - HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


Uber was founded in 2009 and incorporated as Ubercab, Inc. in July 2010, later changing its name to Uber Technologies, Inc. in February 2011. Its principal executive offices are located at 1515 3rd Street, San Francisco, California 94158, and its telephone number is (415) 612-8582.

The company has a partnership with Arizona State University, which has enabled over 4,000 Drivers and their family members to enroll in an undergraduate degree program. Uber is committed to engaging with Drivers and Merchants by listening to their ideas and concerns, piloting innovative dialogue methods, holding regular meetings with Driver associations, and conducting surveys to gather feedback on its app and support services.

Uber operates within a complex and fragmented regulatory environment. Many states and municipalities have adopted Transportation Network Company (TNC) regulations that cover areas such as background checks, vehicle inspections, safety, insurance, and driver conduct. The classificati